In [ ]:
!pip install -U crawl4ai

In [ ]:
!crawl4ai-setup

[INIT].... → Running post-installation setup...
[INIT].... → Installing Playwright browsers...
Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does no

In [ ]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check...
[INIT].... → Crawl4AI 0.4.248
[TEST].... ℹ Testing crawling capabilities...
[EXPORT].. ℹ Exporting PDF and taking screenshot took 2.51s
[FETCH]... ↓ https://crawl4ai.com... | Status: True | Time: 4.04s
[SCRAPE].. ◆ Processed https://crawl4ai.com... | Time: 110ms
[COMPLETE] ● https://crawl4ai.com... | Status: True | Total: 4.16s
[COMPLETE] ● ✅ Crawling test passed!


In [ ]:
!pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"

In [ ]:
## LLM and Api key
local_llm= 'deepseek/deepseek-chat'
base_url = "https://openrouter.ai/api/v1"
key='API_KEY'

In [ ]:
#Initial URL Generation from News Portal
import asyncio
import nest_asyncio
nest_asyncio.apply()
from crawl4ai import *

URLS = []
async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://www.india.gov.in/news_lists",
        )
        for i in result.links['external']:
            print(i['href'])
            URLS.append(i['href'])
print(URLS)

if __name__ == "__main__":
    asyncio.run(main())

[]
[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://www.india.gov.in/news_lists... | Status: True | Time: 0.03s
[COMPLETE] ● https://www.india.gov.in/news_lists... | Status: True | Total: 0.06s
https://www.facebook.com/nationalportalofindia
https://twitter.com/indiagovin
javascript:;
javascript:void(0);
https://www.xn--i1bj3fqcyde.xn--11b7cb3a6a.xn--h2brj9c/news_lists
https://indiacode.nic.in/
javascript:void(0)
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105216
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105215
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105209
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105198
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105196
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105144
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105137
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105136
https://pib.gov.in/PressReleaseIframePage.aspx?PRID=2105131
https://pib.gov.in/PressReleaseIframePage.

In [ ]:
from crawl4ai.content_filter_strategy import PruningContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
Data = []
async def crawl_batch():
    prune_filter = PruningContentFilter(
            # Lower → more content retained, higher → more content pruned
            threshold=0.45,
            # "fixed" or "dynamic"
            threshold_type="dynamic",
            # Ignore nodes with <5 words
            min_word_threshold=5
        )

        # Step 2: Insert it into a Markdown Generator
    md_generator = DefaultMarkdownGenerator(content_filter=prune_filter)
    browser_config = BrowserConfig(headless=True, verbose=False)
    run_config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        stream=False,  # Default: get all results at once
        markdown_generator=md_generator
    )

    dispatcher = MemoryAdaptiveDispatcher(
        memory_threshold_percent=70.0,
        check_interval=1.0,
        max_session_permit=3,
        monitor=CrawlerMonitor(
            display_mode=DisplayMode.DETAILED
        )
    )
    URLS2= []
    for url in URLS:
        if len(url) > 50:
            URLS2.append(url)
    URLS2 = sorted(URLS2, key=len, reverse=True)
    print(URLS2)
    async with AsyncWebCrawler(config=browser_config) as crawler:
        # Get all results at once
        results = await crawler.arun_many(
            urls=URLS2[:10],
            config=run_config,
            dispatcher=dispatcher
        )

        # Process all results after completion
        for result in results:
            if result.success:
                Data.append(result.markdown_v2.fit_markdown)
            else:
                print(f"Failed to crawl {result.url}: {result.error_message}")

if __name__ == "__main__":
    asyncio.run(crawl_batch())

['https://www.newsonair.gov.in/government-open-to-discussion-with-protesting-junior-doctors-demanding-justice-for-kolkata-rape-and-murder-case-mamata-banerjee/', 'https://www.newsonair.gov.in/20-kilometre-relaxation-under-global-navigation-satellite-system-is-not-new-anurag-jain-secretary-of-road-transport-highways/', 'https://ddnews.gov.in/en/world-day-of-social-justice-rs-13611-crores-allocated-to-the-ministry-of-social-justice-and-empowerment-in-union-budget-2025-26/', 'https://ddnews.gov.in/en/hong-kong-and-uyghur-communities-in-ireland-urge-government-to-confront-china-on-transnational-repression-during-wang-yis-visit/', 'https://www.newsonair.gov.in/chandrababu-naidu-welcomes-cabinets-decision-to-extend-₹5-lakh-health-insurance-scheme-to-senior-citizen-above-70-years/', 'https://www.newsonair.gov.in/mp-cm-dr-mohan-yadav-directs-senior-officers-to-refrain-from-taking-leave-due-to-ongoing-excessive-rainfall-and-floods/', 'https://ddnews.gov.in/en/delhi-cm-rekha-gupta-chairs-first-c

Output()

In [ ]:
for i in Data:
    print(i)

  1. **World Day of Social Justice – Rs. 13,611 crore allocated to the Ministry of Social Justice and Empowerment in Union Budget 2025-26**


20/02/25 | 12:33 pm [ | Law and Social Justice](https://ddnews.gov.in/en/world-day-of-social-justice-rs-13611-crores-allocated-to-the-ministry-of-social-justice-and-empowerment-in-union-budget-2025-26/<https:/ddnews.gov.in/en/tag/law-and-social-justice/> "Politics") [ | Ministry of Social Justice and Empowerment](https://ddnews.gov.in/en/world-day-of-social-justice-rs-13611-crores-allocated-to-the-ministry-of-social-justice-and-empowerment-in-union-budget-2025-26/<https:/ddnews.gov.in/en/tag/ministry-of-social-justice-and-empowerment/> "Politics")
## World Day of Social Justice – Rs. 13,611 crore allocated to the Ministry of Social Justice and Empowerment in Union Budget 2025-26
The World Day of Social Justice is observed on February 20 by the United Nations to promote efforts in addressing poverty, exclusion, and unemployment while fostering sol

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_core.documents import Document

docs = []
for doc_raw in Data:
    docs.append(Document(
    page_content=doc_raw,
    metadata={"source": "local"}  # Optional metadata (e.g., source, author)
    ))
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="latest-news",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", nomic_api_key="API_KEY",inference_mode='remote')
)
retriever = vectorstore.as_retriever()

In [ ]:
!pip install langchain_openai

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain.chat_models import ChatOpenAI
# LLM

llm = ChatOpenAI(model=local_llm,base_url=base_url,api_key=key, temperature=0)

prompt = PromptTemplate(
    template="""You are an expert at routing a user question to a vectorstore or web search. \n
    Use the vectorstore for questions on {content}. \n
    You do not need to be stringent with the keywords in the question related to these topics. \n
    Otherwise, use web-search. Give a binary choice 'web_search' or 'vectorstore' based on the question. \n
    Return the a JSON with a single key 'datasource' and no premable or explanation. \n
    Question to route: {question}""",
    input_variables=["question","content"],
)

question_router = prompt | llm | JsonOutputParser()
question = "new Income Tax bill in Lok Sabha"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question,"content":doc_txt}))

{'datasource': 'web_search'}


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

# LLM
llm = ChatOpenAI(model=local_llm,base_url=base_url,api_key=key, temperature=0)


prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "lok sabha"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


In [ ]:
### Generate

from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model=local_llm,base_url=base_url,api_key=key, temperature=1)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "What is the new income tax bill in Lok Sabha? in detail"
generation = rag_chain.invoke({"context": doc_txt, "question": question})
print(generation)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


The context provided does not mention any details about a new income tax bill in Lok Sabha. Therefore, I don't have the information to answer your question.


In [ ]:
### Hallucination Grader

# LLM
llm = ChatOpenAI(model=local_llm,base_url=base_url,api_key=key, temperature=1)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is grounded in / supported by a set of facts. \n
    Here are the facts:
    \n ------- \n
    {documents}
    \n ------- \n
    Here is the answer: {generation}
    Give a binary score 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})


{'score': 'yes'}

In [ ]:
### Answer Grader

# LLM
llm = ChatOpenAI(model=local_llm,base_url=base_url,api_key=key, temperature=1)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is useful to resolve a question. \n
    Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question}
    Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})


{'score': 'no'}

In [ ]:
### Question Re-writer

# LLM
llm = ChatOpenAI(model=local_llm,base_url=base_url,api_key=key, temperature=1)

# Prompt
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n {question}. Improved question with no preamble: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'Can you provide a detailed overview of the new income tax bill recently introduced in the Lok Sabha, including its key provisions, implications, and potential impact on taxpayers?'

# Graph

In [ ]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]


In [ ]:
from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    # print("---RETRIEVE---")
    print("Reteriving Data from Vector DataBase")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("Generating Answer")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("Check if question is relevant to vector database")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        if grade == "yes":
            print("Relevant Document")
            # print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("Document Not relevant")
            # print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("Transforming Question")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import getpass
os.environ["TAVILY_API_KEY"]='API_KEY'
web_search_tool = TavilySearchResults(k=3)

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    print(docs)
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}

def web_search_Store(state):
    """
    Stores the Web search based.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("Storing WEB Search")
    question = state["question"]
    docs = state["documents"]
    vectorstore.add_documents(
    documents=[docs]
    )
    print("---STORED DATA IN VECTOR DATA----")
    web_results = docs
    return {"documents": web_results, "question": question}


In [ ]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("Routing Question")
    question = state["question"]
    print(question)
    docs = retriever.get_relevant_documents(question)
    doc_txt = docs[1].page_content
    source = question_router.invoke({"question": question,'content':doc_txt})
    print(source)
    print(source["datasource"])
    if source["datasource"] == "web_search":
        print("Searching Question Online")
        return "web_search"
    elif source["datasource"] == "vectorstore":
        print("Searching Question in vector database")
        return "vectorstore"
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("Checking Question if it is properly formated")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("Question is not properly asked | reformatting")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("Checking for hallucination")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"



In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("store_web_search", web_search_Store)
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("web_search", "store_web_search")
workflow.add_edge("store_web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search")
workflow.add_edge("web_search", "store_web_search")
workflow.add_edge("store_web_search", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()


In [ ]:
from pprint import pprint
import time

# Run
inputs = {"question": "what is self attention in RAG?"}
for output in app.stream(inputs):
    time.sleep(5)
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])


Routing Question
what is self attention in RAG?
{'datasource': 'web_search'}
web_search
Searching Question Online
---WEB SEARCH---
[{'url': 'https://medium.com/@ramendrakumar/self-attention-d8196b9e9143', 'content': 'The primary function of self-attention is to generate context-aware vectors from the input sequence itself rather than considering both input and output as in the RNN-based encoder-decoder architecture. Let’s understand the above paragraph by writing a mathematical equation as given below, where A(i), α(i,j) are weighted sum (termed as attention vector) and attention scores (weights) respectively, and individual word vector is represented by x_j. In Self-Attention, we get rid of RNN units and calculate the context-aware vectors from the input sequence itself. In self-attention, every word acts as a query once, while the entire words in the sequence act as keys, and attention weights are calculated to figure out which key matches the query.'}, {'url': 'https://medium.com/@k